# Agent faas deployment with hydraa faas manager

This notebook demonstrates function deployment using the hydraa faas manager with agent faas middleware. it covers two primary methods:
1. **inline code**: defining function code directly within the notebook.
2. **project directories**: deploying functions from project directories.

## Prerequisites
- agent faas middleware running and configured (on localhost:8000 by default)
- hydraa faas manager installed
- python environment with necessary packages (`pip install -e .`)

In [ ]:
from src.hydraa_faas.faas_manager.manager import FaasManager
from hydraa.services import ServiceManager
from hydraa import Task
import time
import shutil
import json
import requests

## Initial setup

Configure the connection to the agent, check its health, and initialize the faas manager.

In [ ]:
agent_config = {
    'host': 'localhost',
    'port': 8000,
}
agent_url = f"http://{agent_config['host']}:{agent_config['port']}"

print("checking agent faas middleware health...")
try:
    response = requests.get(f"{agent_url}/health", timeout=5)
    agent_healthy = response.json().get('status') == 'healthy'
except Exception as e:
    agent_healthy = False

if agent_healthy:
    print("agent is healthy and ready.")
else:
    print("agent is not healthy. please ensure the agent is running before continuing.")

if agent_healthy:
    faas_mgr = FaasManager(proxy_mgr=None, asynchronous=True, auto_terminate=True, agent_config=agent_config)
    service_mgr = ServiceManager([faas_mgr])
    service_mgr.start_services()
    print("faas manager started.")

## Example 1: deploying an inline function

This example defines the function's source code directly in the notebook.

In [ ]:
inline_task = Task()
inline_task.provider = 'agent'
inline_task.handler_code = '''
import json
def handler(event, context=None):
    name = event.get('name', 'world')
    return {'statusCode': 200, 'body': json.dumps({'message': f'hello, {name} from an inline function!'})}
'''
inline_task.handler = 'main.handler'
inline_task.runtime = 'python3.9'

print("submitting inline function for deployment")
faas_mgr.submit(inline_task)

## Example 2: deploying from project files

This example deploys functions from project directories. Remember to run the example directory creation script first.

In [ ]:
# create a task for the calculator function zip
calculator_task = Task()
calculator_task.provider = 'agent'
calculator_task.source_path = '.agent_functions/calculator.zip'
calculator_task.handler = 'main.handler'
calculator_task.runtime = 'python3.9'

# create a task for the text analyzer function zip
text_analyzer_task = Task()
text_analyzer_task.provider = 'agent'
text_analyzer_task.source_path = './agent_functions/text_analyzer'
text_analyzer_task.handler = 'main.handler'
text_analyzer_task.runtime = 'python3.9'

zip_tasks = [calculator_task, text_analyzer_task]

print("submitting zipped functions for deployment")
faas_mgr.submit(zip_tasks)

## Get deployment results

Wait for all submitted functions to finish deploying.

In [ ]:
all_tasks = [inline_task] + zip_tasks
deployed_functions = []

print("waiting for all deployments to complete...")
for task in all_tasks:
    try:
        result = task.result(timeout=300)
        print(f"success: {task.name} deployed. result: {result}")
        deployed_functions.append(task)
    except Exception as e:
        print(f"failed to deploy a function. error: {e}")

## Cleanup

Shut down the faas manager to clean up deployed functions.

In [ ]:
print("shutting down services and cleaning up resources")
if 'service_mgr' in locals():
    service_mgr.shutdown_services()
shutil.rmtree('./examples/agent_functions')
print("cleanup complete.")